<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Feature Engineering: Polynomials and Interactions
              
</p>
</div>

Data Science Cohort Live NYC June 2022
<p>Phase 2: Topic 20</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

#### Feature Engineering: Transforming input data
- Input in form that the better conforms to structure of input-output relationship.
- Model interactions between features.

#### A key aspect to making a better prediction machine

- Target may not depend linearly on feature set.
- I.e. cant approximate well by a hyperplane.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
noise = norm.rvs(scale = 16, size = 400)
x = np.linspace(-7,7, 400) + 300
y = 3*(x-300)**2  + 4*(x-300) + 100 + noise
tempyield_data = pd.DataFrame({'Temp': x, 'Yield': y})

In [ ]:
%%capture yieldvtemp
sns.set_context('talk')
fig, ax = plt.subplots()
sns.scatterplot(y ='Yield', x = 'Temp', data = tempyield_data, ax = ax)
ax.set_title('Yield vs. Temperature')
plt.show()

In [ ]:
yieldvtemp()

Fitting yield to a model simply linear in temperature might be considered silly. 

What kind of dependence might we try?

In [ ]:
tempyield_data.head()

Perform a regression:
$$ Y = \beta_2T^2 + \beta_1T + \beta_0  $$

*Note*: this is still a model **linear** in coefficients. 

Construct another column that is $T^2$.

In [ ]:
tempyield_data['Temp_squared'] = tempyield_data['Temp']**2
tempyield_data.head()

Linear regression of Yield on $T$ and $T^2$.

Doing a regression just on T. Silly but lets do it anyway to see:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:

X = tempyield_data['Temp']
y = tempyield_data['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

lr_simple = LinearRegression()
lr_simple.fit(X_train.values.reshape(-1,1), y_train)

In [ ]:
print(lr_simple.coef_, lr_simple.intercept_)

In [ ]:
lr_simple.score(X_train.values.reshape(-1,1),
                y_train)

In [ ]:
y_pred = lr_simple.predict(X_test.values.reshape(-1,1))
mean_absolute_error(y_pred, y_test)

But saying there is no relation between temperature and yield is obviously a mistake!

Regression on $T$ and $T^2$.

In [ ]:
X_poly = tempyield_data[['Temp', 'Temp_squared']]
y = tempyield_data['Yield']

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size = 0.3)

lr_poly = LinearRegression()
lr_poly.fit(X_train, y_train)

In [ ]:
print(lr_poly.coef_, lr_poly.intercept_)

In [ ]:
lr_poly.score(X_train, y_train)

In [ ]:
y_pred_poly = lr_poly.predict(X_test)
mean_absolute_error(y_pred_poly, y_test)

Let's visualize the difference in predictions:

In [ ]:
beta_1, beta_2 = lr_poly.coef_
beta_0 = lr_poly.intercept_

poly_pred = beta_2*X**2 + beta_1*X + beta_0

fig, ax = plt.subplots()
ax.plot(X, poly_pred, c ='r')
ax.scatter(X, y)
plt.show()

In [ ]:
beta_1 = lr_simple.coef_[0]
beta_0 = lr_simple.intercept_

lin_pred = beta_1*X + beta_0

fig, ax = plt.subplots()
ax.plot(X, lin_pred, c ='r')
ax.scatter(X, y)
plt.show()

#### More generally

Target depend on feature with higher order polynomial. Procedure would be same.

$$ Y = \beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_n x^n $$

Again:
- Construct feature columns for each power of x.
- Fit $\beta_i$ using linear regression.

#### Using scikit-learn: PolynomialFeatures with a single feature

- With single feature is easy/fast way to construct these features at higher power.
- Again, main advantage: can be used in a pipeline.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

PolynomialFeatures()
- Takes arguments for highest order of polynomial
- Can chose to include zeroth power term for fitting bias (useful for statsmodels)

In [ ]:
pf = PolynomialFeatures(degree = 3, include_bias = False )
X_to_second = pf.fit_transform(X.values.reshape(-1,1))
X_to_second[0:10]

And fitting is exactly the same as before.

#### PolynomialFeatures() with multiple features.

In [ ]:
sales_df = pd.read_csv('Data/advertising.csv').drop(columns = ['Unnamed: 0'])
sales_df.head()

In [ ]:
sales_df.info()

In [ ]:
sns.heatmap(sales_df.corr(), annot = True)

Maybe want to model interactions

Recall that $$ Cov(Radio, Sales) \rightarrow Radio\times Sales $$

Average of product encodes correlation between the two.

Using products of features as new features:

- Factors pairwise correlations of original features into prediction.

Model:
$$ Sales = \beta_{TV} TV + \beta_{Radio} Radio + \beta_{Newspaper} Newspaper + \\ \beta^{int}_1 \Big( TV \times Radio \Big) + \beta^{int}_2 \Big( TV \times Newspaper \Big) + \beta^{int}_3 \Big(Radio \times Newspaper\Big) $$

In [ ]:
X = sales_df[['TV', 'radio', 'newspaper']]
y = sales_df['sales']

PolynomialFeatures:
- has argument interaction_only = True/False.
- If true only takes interactions terms at highest order specified.

In [ ]:
linear_withinteraction = PolynomialFeatures(degree = 2, interaction_only = True, include_bias = False)

In [ ]:
X_transformed = linear_withinteraction.fit_transform(X)
feat_names = pd.Series(linear_withinteraction.get_feature_names())
feat_names

In [ ]:
map_dict = {'x0': 'TV', 'x1': 'Radio', 'x2': 'Newspaper'}
for key, value in map_dict.items():
    feat_names = feat_names.str.replace(key, value)

X_trans_df = pd.DataFrame(X_transformed)
X_trans_df.columns = feat_names

After pandafying our numpy array:

In [ ]:
X_trans_df.head()

Looks good. Let's standardize the variables and analyze the weights of the factors and their interactions.

In [ ]:
X_trans_standard = X_trans_df.apply(lambda x: (x - x.mean())/x.std() )
y_standard = (y - y.mean())/y.std()

In [ ]:
import statsmodels.api as sm
results = sm.OLS(y_standard, X_trans_standard).fit()

In [ ]:
results.summary()

In [ ]:
trans_with_inter = X_trans_standard.loc[:,['TV', 'Radio', 'TV Radio']]
trans_with_inter['Sales'] = y_standard

sns.pairplot(trans_with_inter)

Might do even better by modeling the knees in the feature set and constructing a new feature.
- Splines
- Trend filters
- etc.

Look it up.

Some words of advice:
- Adding interactions and higher order terms can be useful.
- But they can also add extra variables that can lead to overfitting or useless parameters.
- Be careful and use your judgment.

#### Also: interaction terms not always interpretable. But may lead to better predictions.

Takeaway:
- This is where EDA is really important:
    - Visualization
    - Descriptive statistics 
    - Correlation matrices
    - Playing with your data!
    
See what transformations and functional relations useful for model inference and prediction.

Be the sculptor.

<center><img src = "Images/rodin_sculptor.jpg" width = 500/></center>
<center>Rodin in his nightie.</center>